In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/customer-churn-prediction-2020/sampleSubmission.csv
/kaggle/input/customer-churn-prediction-2020/train.csv
/kaggle/input/customer-churn-prediction-2020/test.csv


In [3]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV

In [4]:
# Load training and testing data from google drive.
df_train = pd.read_csv('/kaggle/input/customer-churn-prediction-2020/train.csv')
df_test = pd.read_csv('/kaggle/input/customer-churn-prediction-2020/test.csv')

In [5]:
# View training data rows.
df_train.head()

,state,account_length,area_code,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,churn
0,OH,107,area_code_415,no,yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,no
1,NJ,137,area_code_415,no,no,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,no
2,OH,84,area_code_408,yes,no,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,no
3,OK,75,area_code_415,yes,no,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,no
4,MA,121,area_code_510,no,yes,24,218.2,88,37.09,348.5,108,29.62,212.6,118,9.57,7.5,7,2.03,3,no


In [6]:
df_test.head()

,id,state,account_length,area_code,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls
0,1,KS,128,area_code_415,no,yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1
1,2,AL,118,area_code_510,yes,no,0,223.4,98,37.98,220.6,101,18.75,203.9,118,9.18,6.3,6,1.70,0
2,3,IA,62,area_code_415,no,no,0,120.7,70,20.52,307.2,76,26.11,203.0,99,9.14,13.1,6,3.54,4
3,4,VT,93,area_code_510,no,no,0,190.7,114,32.42,218.2,111,18.55,129.6,121,5.83,8.1,3,2.19,3
4,5,NE,174,area_code_415,no,no,0,124.3,76,21.13,277.1,112,23.55,250.7,115,11.28,15.5,5,4.19,3


In [7]:
# Data Preprocessing

# Frequency Encoding for 'state'
frq_dis = df_train.groupby('state').size()/len(df_train)
df_train["state"] = df_train.state.map(frq_dis)
frq_dis = df_test.groupby('state').size()/len(df_test)
df_test["state"] = df_test.state.map(frq_dis)

In [8]:
df_train.head()

,state,account_length,area_code,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,churn
0,0.022353,107,area_code_415,no,yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,no
1,0.022588,137,area_code_415,no,no,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,no
2,0.022353,84,area_code_408,yes,no,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,no
3,0.018353,75,area_code_415,yes,no,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,no
4,0.020941,121,area_code_510,no,yes,24,218.2,88,37.09,348.5,108,29.62,212.6,118,9.57,7.5,7,2.03,3,no


In [9]:
df_test.head()

,id,state,account_length,area_code,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls
0,1,0.016000,128,area_code_415,no,yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1
1,2,0.030667,118,area_code_510,yes,no,0,223.4,98,37.98,220.6,101,18.75,203.9,118,9.18,6.3,6,1.70,0
2,3,0.009333,62,area_code_415,no,no,0,120.7,70,20.52,307.2,76,26.11,203.0,99,9.14,13.1,6,3.54,4
3,4,0.020000,93,area_code_510,no,no,0,190.7,114,32.42,218.2,111,18.55,129.6,121,5.83,8.1,3,2.19,3
4,5,0.020000,174,area_code_415,no,no,0,124.3,76,21.13,277.1,112,23.55,250.7,115,11.28,15.5,5,4.19,3


In [10]:
# One-hot Encoding for 'area_code'
enc_one = OneHotEncoder()
x = df_train["area_code"].values.reshape(-1,1)
val_train = enc_one.fit_transform(x).toarray()
df_onehot_train = pd.DataFrame(val_train,columns=['is_'+str(enc_one.categories_[0][i]) for i in range(len(enc_one.categories_[0]))])
df_enc_train = pd.concat([df_onehot_train, df_train], axis=1)
df_enc_train.drop("area_code", inplace=True, axis=1)

In [11]:
df_enc_train.head()

,is_area_code_408,is_area_code_415,is_area_code_510,state,account_length,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,...,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,churn
0,0.0,1.0,0.0,0.022353,107,no,yes,26,161.6,123,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,no
1,0.0,1.0,0.0,0.022588,137,no,no,0,243.4,114,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,no
2,1.0,0.0,0.0,0.022353,84,yes,no,0,299.4,71,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,no
3,0.0,1.0,0.0,0.018353,75,yes,no,0,166.7,113,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,no
4,0.0,0.0,1.0,0.020941,121,no,yes,24,218.2,88,...,108,29.62,212.6,118,9.57,7.5,7,2.03,3,no


In [12]:
x = df_test["area_code"].values.reshape(-1,1)
val_test = enc_one.fit_transform(x).toarray()
df_onehot_test = pd.DataFrame(val_test,columns=['is_'+str(enc_one.categories_[0][i]) for i in range(len(enc_one.categories_[0]))])
df_enc_test = pd.concat([df_onehot_test, df_test], axis=1)
df_enc_test.drop(["area_code","id"], inplace=True, axis=1)

In [13]:
df_enc_test.head()

,is_area_code_408,is_area_code_415,is_area_code_510,state,account_length,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,...,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls
0,0.0,1.0,0.0,0.016000,128,no,yes,25,265.1,110,...,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1
1,0.0,0.0,1.0,0.030667,118,yes,no,0,223.4,98,...,220.6,101,18.75,203.9,118,9.18,6.3,6,1.70,0
2,0.0,1.0,0.0,0.009333,62,no,no,0,120.7,70,...,307.2,76,26.11,203.0,99,9.14,13.1,6,3.54,4
3,0.0,0.0,1.0,0.020000,93,no,no,0,190.7,114,...,218.2,111,18.55,129.6,121,5.83,8.1,3,2.19,3
4,0.0,1.0,0.0,0.020000,174,no,no,0,124.3,76,...,277.1,112,23.55,250.7,115,11.28,15.5,5,4.19,3


In [14]:
# Label Encoding for 'international_plan' and 'voice_mail_plan'
le = LabelEncoder()
df_enc_train['international_plan'] = le.fit_transform(df_enc_train['international_plan'])
df_enc_test['international_plan'] = le.transform(df_enc_test['international_plan'])
df_enc_train['voice_mail_plan'] = le.fit_transform(df_enc_train['voice_mail_plan'])
df_enc_test['voice_mail_plan'] = le.transform(df_enc_test['voice_mail_plan'])

In [15]:
df_enc_train['international_plan'].value_counts()

international_plan
0    3854
1     396
Name: count, dtype: int64

In [16]:
df_enc_test['international_plan'].value_counts()

international_plan
0    673
1     77
Name: count, dtype: int64

In [17]:
df_enc_train['voice_mail_plan'].value_counts()

voice_mail_plan
0    3138
1    1112
Name: count, dtype: int64

In [18]:
df_enc_test['voice_mail_plan'].value_counts()

voice_mail_plan
0    539
1    211
Name: count, dtype: int64

In [19]:
df_enc_train.head()

,is_area_code_408,is_area_code_415,is_area_code_510,state,account_length,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,...,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,churn
0,0.0,1.0,0.0,0.022353,107,0,1,26,161.6,123,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,no
1,0.0,1.0,0.0,0.022588,137,0,0,0,243.4,114,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,no
2,1.0,0.0,0.0,0.022353,84,1,0,0,299.4,71,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,no
3,0.0,1.0,0.0,0.018353,75,1,0,0,166.7,113,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,no
4,0.0,0.0,1.0,0.020941,121,0,1,24,218.2,88,...,108,29.62,212.6,118,9.57,7.5,7,2.03,3,no


In [28]:
df_enc_test.head()

,is_area_code_408,is_area_code_415,is_area_code_510,state,account_length,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,...,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls
0,0.0,1.0,0.0,0.016000,128,0,1,25,265.1,110,...,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1
1,0.0,0.0,1.0,0.030667,118,1,0,0,223.4,98,...,220.6,101,18.75,203.9,118,9.18,6.3,6,1.70,0
2,0.0,1.0,0.0,0.009333,62,0,0,0,120.7,70,...,307.2,76,26.11,203.0,99,9.14,13.1,6,3.54,4
3,0.0,0.0,1.0,0.020000,93,0,0,0,190.7,114,...,218.2,111,18.55,129.6,121,5.83,8.1,3,2.19,3
4,0.0,1.0,0.0,0.020000,174,0,0,0,124.3,76,...,277.1,112,23.55,250.7,115,11.28,15.5,5,4.19,3


In [29]:
# Normalization
scaler = MinMaxScaler()
numerical_features = df_enc_train.select_dtypes(include=['float64', 'int64']).columns
df_enc_train[numerical_features] = scaler.fit_transform(df_enc_train[numerical_features])
df_enc_test[numerical_features] = scaler.transform(df_enc_test[numerical_features])

In [30]:
df_enc_train.head()

,is_area_code_408,is_area_code_415,is_area_code_510,state,account_length,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,...,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,churn
0,0.0,1.0,0.0,0.56,0.438017,0.0,1.0,0.500000,0.459744,0.745455,...,0.605882,0.544204,0.644051,0.588571,0.644344,0.685,0.15,0.685185,0.111111,no
1,0.0,1.0,0.0,0.57,0.561983,0.0,0.0,0.000000,0.692461,0.690909,...,0.647059,0.337263,0.411646,0.594286,0.411930,0.610,0.25,0.609259,0.000000,no
2,1.0,0.0,0.0,0.56,0.342975,1.0,0.0,0.000000,0.851778,0.430303,...,0.517647,0.172233,0.498481,0.508571,0.498593,0.330,0.35,0.329630,0.222222,no
3,0.0,1.0,0.0,0.39,0.305785,1.0,0.0,0.000000,0.474253,0.684848,...,0.717647,0.412901,0.473165,0.691429,0.473270,0.505,0.15,0.505556,0.333333,no
4,0.0,0.0,1.0,0.50,0.495868,0.0,1.0,0.461538,0.620768,0.533333,...,0.635294,0.969876,0.538228,0.674286,0.538548,0.375,0.35,0.375926,0.333333,no


In [31]:
df_enc_test.head()

,is_area_code_408,is_area_code_415,is_area_code_510,state,account_length,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,...,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls
0,0.0,1.0,0.0,0.290000,0.524793,0.0,1.0,0.480769,0.754196,0.666667,...,0.549402,0.582353,0.549443,0.619494,0.520000,0.619584,0.500,0.15,0.500000,0.111111
1,0.0,0.0,1.0,0.913333,0.483471,1.0,0.0,0.000000,0.635562,0.593939,...,0.613972,0.594118,0.613949,0.516203,0.674286,0.516601,0.315,0.30,0.314815,0.000000
2,0.0,1.0,0.0,0.006667,0.252066,0.0,0.0,0.000000,0.343385,0.424242,...,0.854996,0.447059,0.854944,0.513924,0.565714,0.514350,0.655,0.30,0.655556,0.444444
3,0.0,0.0,1.0,0.460000,0.380165,0.0,0.0,0.000000,0.542532,0.690909,...,0.607292,0.652941,0.607400,0.328101,0.691429,0.328081,0.405,0.15,0.405556,0.333333
4,0.0,1.0,0.0,0.460000,0.714876,0.0,0.0,0.000000,0.353627,0.460606,...,0.771222,0.658824,0.771120,0.634684,0.657143,0.634778,0.775,0.25,0.775926,0.333333


In [32]:
# Model Training and Hyperparameter Tuning
X_train = df_enc_train.drop('churn', axis=1)
y_train = df_enc_train['churn']

In [33]:
# Decision Tree with Hyperparameter Tuning
param_grid = {'max_depth': [3, 5, 7, 10], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]}
grid_search = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=10, scoring='accuracy')
grid_search.fit(X_train, y_train)
best_dt = grid_search.best_estimator_

In [34]:
# Evaluate the optimized model
k_fold = KFold(n_splits=10, shuffle=True, random_state=0)
dt_scores = cross_val_score(best_dt, X_train, y_train, cv=k_fold, scoring='accuracy')
mean_score_dt = dt_scores.mean()
print(f"The mean score of cross validation using the optimized Decision Tree algorithm is {mean_score_dt * 100:.2f}%")

The mean score of cross validation using the optimized Decision Tree algorithm is 94.61%


In [35]:
# Prediction
best_dt.fit(X_train, y_train)
X_test = df_enc_test
predictions = best_dt.predict(X_test)

In [36]:
# Create submission file
data_test = pd.read_csv('/kaggle/input/customer-churn-prediction-2020/test.csv')
submission = pd.DataFrame({
        "id": data_test["id"],
        "churn": predictions
    })

submission.head()

,id,churn
0,1,no
1,2,no
2,3,yes
3,4,no
4,5,no


In [37]:
submission.to_csv('/kaggle/working/Prediction_result1.csv', index=False)

print("Submission file created successfully.")

Submission file created successfully.
